# Connectivity Classifier

This notebook trains a binary classifier to predict the group of a participant (AVGP or NVGP) based on their connectivity matrices.

It implements the following steps:

1. Load the data
2. Cross-validated classification
3. Permutation testing
4. Permutation importance
5. SHAP


## Inputs

Connectivity matrices including five connectivity metrics (correlation, partial correlation, tangent, precision, and covariance), five parcellations (DiFuMo64, Dosenbach2010, Gordon2014, Friedman2020, and Seitzman2018), and two aggregation mode (reigon-level and network-level).

## Outputs

Prediction accuracies on the test set for each combination of connectivity metric, parcellation, and aggregation mode. The results are stored in the following file:
  - `models/connectivity_classifier_*.nc5`


## Requirements

To run this notebook, you need to have a few packages installed. The easiest way to do this is to use mamba to create a new environment from the `environment.yml` file in the root of this repository:

```bash
mamba env create -f environment.yml

# Then, activate the `acnets` environment:
mamba activate acnets
```

In [17]:
# 0. SETUP

%reload_ext autoreload
%autoreload 3

import pandas as pd
from pathlib import Path
import xarray as xr
import scipy.stats as stats
import numpy as np
from src.acnets.pipeline import ConnectivityPipeline, ConnectivityVectorizer
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from sklearn.inspection import permutation_importance
from sklearn.model_selection import (GridSearchCV, StratifiedShuffleSplit,
                                     cross_val_score, learning_curve,
                                     permutation_test_score)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import LinearSVC
from tqdm.auto import tqdm
from IPython.display import clear_output

In [18]:
# PARAMETERS

CV = StratifiedShuffleSplit(n_splits=10, test_size=8)
N_PERMUTATIONS = 10
N_TOP_MODELS = 10  # number of top models to run permutation test, SHAP, etc.
AGGREGATE_NETWORKS = False  # whether aggregate regions into networks or not
ENABLE_SHAP = False  # whether to run SHAP analysis or not as it takes a long time

MODELS_DIR= Path('models')

In [19]:
# DATA PREPARATION

subjects = ConnectivityPipeline().transform('all').coords['subject'].values
groups = [s[:4] for s in subjects]  # AVGP or NVGP

X = subjects.reshape(-1, 1)

y_encoder = LabelEncoder()
y = y_encoder.fit_transform(groups)

In [20]:
# PREPARE OUTPUT DATASET FILE

n_splits = CV.get_n_splits()
n_folds = int(X.shape[0] / CV.test_size)

model_output_name = ('connectivities'
                     '_classifier-SVM'
                     '_measure-accuracy'
                     f'_shap-{"enabled" if ENABLE_SHAP else "disabled"}'
                     f'_agg-{"network" if AGGREGATE_NETWORKS else "region"}'
                     f'_top-{N_TOP_MODELS}'
                     f'_cv-{n_splits}x{n_folds}fold.nc5'
                     )

OUTPUT_PATH = MODELS_DIR / model_output_name

In [21]:
# DEFINE PIPELINE

from sklearn.svm import SVC

pipe  = Pipeline([
    ('connectivity', ConnectivityPipeline(atlas='friedman2020', kind='partial correlation', agg_networks=False)),
    ('vectorize', ConnectivityVectorizer()),
    ('scale', StandardScaler()),
    ('zerovar', VarianceThreshold()),
    ('select', SelectFromModel(LinearSVC(penalty='l1', dual=False, max_iter=10000),
                               max_features=lambda x: min(10, x.shape[1]))),
    ('clf', LinearSVC(penalty='l1', dual=False, max_iter=10000))
    # ('clf', SVC(kernel='linear', C=1))
])

# DEBUG (expected to overfit, i.e., score=1)
pipe.fit(X, y).score(X, y)

1.0

In [22]:
# VERIFY THE MODEL (calculate cross-validated accuracy and bootstrap CI)
pipe.set_params(connectivity__atlas='friedman2020',
                connectivity__kind='covariance',
                connectivity__agg_networks=False)

scores = cross_val_score(pipe, X, y,
                         cv=CV,
                         scoring='accuracy',
                         n_jobs=-1)
bootstrap_ci = stats.bootstrap(scores.reshape(1,-1), np.mean)

print('Test accuracy (mean ± std): {:.2f} ± {:.2f}'.format(scores.mean(), scores.std()))
print(bootstrap_ci.confidence_interval)

Test accuracy (mean ± std): 0.55 ± 0.19
ConfidenceInterval(low=0.4375, high=0.6625)


In [23]:
# RUN PIPELINE ON ALL ATLASES AND METRICS

param_grid = {
    'connectivity__agg_networks': [AGGREGATE_NETWORKS],
    # 'connectivity__atlas': ['gordon2014_2mm', 'dosenbach2010', 'difumo_64_2mm', 'friedman2020'],
    'connectivity__atlas': ['gordon2014_2mm'],
    'connectivity__kind': ['partial correlation', 'correlation', 'covariance', 'precision'],  # 'tangent' ],
}

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=CV,
    verbose=2,
    n_jobs=-2,
    scoring='accuracy')

grid.fit(X, y)
clear_output(wait=True)


print('best estimator:', grid.best_estimator_, '\n', 'best score:', grid.best_score_)


best estimator: Pipeline(steps=[('connectivity',
                 ConnectivityPipeline(atlas='gordon2014_2mm',
                                      kind='correlation',
                                      agg_networks=False,
                                      mock=False,
                                      bids_dir='data/julia2018',
                                      parcellation_cache_dir='data/julia2018/derivatives/resting_timeseries/')),
                ('vectorize', ConnectivityVectorizer()),
                ('scale', StandardScaler()), ('zerovar', VarianceThreshold()),
                ('select',
                 SelectFromModel(estimator=LinearSVC(dual=False, max_iter=10000,
                                                     penalty='l1'),
                                 max_features=<function <lambda> at 0x7fed75b8f370>)),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))]) 
 best score: 0.575


In [24]:
# STORE RESULTS

def generate_model_name(params):
    """Helper function to generate a model name from the parameters."""

    atlas_name = params['connectivity__atlas'].replace('_2mm', '').replace('_','')
    agg_name = 'networks' if params['connectivity__agg_networks'] else 'regions'
    kind_name = params['connectivity__kind'].replace(' ', '-')
    name = f'{atlas_name}_{agg_name}_{kind_name}'

    return name

grid_results = pd.DataFrame(grid.cv_results_)

grid_results['grid_model_name'] = grid_results['params'].apply(generate_model_name)

grid_results.set_index('grid_model_name', inplace=True)
grid_results.drop(columns=['params'], inplace=True)

ds_grid = grid_results.to_xarray()
ds_grid['scoring'] = grid.scoring
ds_grid['cv_test_size'] = CV.test_size
ds_grid['cv_n_splits'] = CV.n_splits
ds_grid['n_subjects'] = len(X)

In [25]:
# PERMUTATION TEST (SHUFFLE Y)


perm_scores_agg = []
cv_scores_agg = []
pvalues = []
model_names = []

# sort by rank and take top models
top_models = pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')[:N_TOP_MODELS].loc[:,'params'].to_list()

for p in (progress_bar := tqdm(top_models)):

    pipe.set_params(**p)

    model_name = generate_model_name(p)
    progress_bar.set_description(model_name)

    _, perm_scores, pvalue = permutation_test_score(pipe, X, y,
                                                    scoring='accuracy',
                                                    n_permutations=N_PERMUTATIONS,
                                                    cv=CV,
                                                    n_jobs=-2,
                                                    verbose=1)

    cv_scores = cross_val_score(pipe, X, y,
                                cv=CV,
                                scoring='accuracy',
                                n_jobs=-2,
                                verbose=0)

    perm_scores_agg.append(perm_scores)
    cv_scores_agg.append(cv_scores)
    pvalues.append(pvalue)
    model_names.append(model_name)

ds_perm_test = xr.Dataset({
    'perm_scores': (('model_name', 'permutation_dim'), perm_scores_agg),
    'cv_scores': (('model_name', 'cv_dim'), cv_scores_agg),
    'pvalue': (('model_name'), pvalues)},
    coords={'model_name': model_names})


  0%|          | 0/4 [00:00<?, ?it/s]

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   8 out of  10 | elapsed:   26.8s remaining:    6.7s
[Parallel(n_jobs=-2)]: Done  10 out of  10 | elapsed:   27.3s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   8 out of  10 | elapsed:   22.8s remaining:    5.7s
[Parallel(n_jobs=-2)]: Done  10 out of  10 | elapsed:   24.2s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   8 out of  10 | elapsed:   36.2s remaining:    9.0s
[Parallel(n_jobs=-2)]: Done  10 out of  10 | elapsed:   38.9s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   8 out of  10 | elapsed:   33.4s remaining:    8.4s
[Parallel(n_jobs=-2)]: Done  10 out of  10 | elapsed:   36.4s finished


In [26]:
# PERMUTATION FEATURE IMPORTANCE (SHUFFLE X)

importances_cv = []

for p in (progress_bar := tqdm(top_models)):

    pipe.set_params(**p)

    model_name = generate_model_name(p)
    progress_bar.set_description(model_name)

    # get feature names for the connectivity vector
    X_conn = pipe[:2].transform(X)
    feature_names = pipe[:2].get_feature_names_out()

    importances = []

    # cross-validated permutation importance
    for train, test in tqdm(CV.split(X,y), total=CV.get_n_splits(X,y), desc='CV', leave=False):
        pipe.fit(X[train], y[train])

        results = permutation_importance(pipe[2:], X_conn[test], y[test],
                                        scoring=grid.scoring,
                                        n_jobs=-1)
        importances.append(results.importances.T)

    feature_dim_name = f'{"_".join(model_name.split("_")[0:2])}_feature'

    importances_cv_dataset = xr.Dataset({
        f'{model_name} importances': (('permutation_importance_num', feature_dim_name), np.vstack(importances))},
        coords={feature_dim_name: feature_names}
    )

    importances_cv.append(importances_cv_dataset)
    
    # sort by mean importance
    importances = pd.DataFrame(np.vstack(importances), columns=feature_names)
    sorted_columns = importances.mean(axis=0).sort_values(ascending=False).index
    importances = importances[sorted_columns]

ds_perm_importance = xr.merge(importances_cv)

  0%|          | 0/4 [00:00<?, ?it/s]

TypeError: sequence item 0: expected str instance, int found

In [ ]:
# SHAP

if not ENABLE_SHAP:
    raise ValueError('ENABLE_SHAP must be True to run SHAP analysis')


import shap
import logging

# turn off shap info-level logs while using progress bars
logging.getLogger('shap').setLevel(logging.WARNING)

shap_agg = []

for p in (progress_bar := tqdm(top_models)):

    pipe.set_params(**p)

    model_name = generate_model_name(p)
    progress_bar.set_description(model_name)

    shap_values_cv = []
    test_indices = []
    y_test_cv = []
    y_pred_cv = []

    feature_names = pipe[:2].get_feature_names_out()

    X_conn = pipe[:2].fit_transform(X, y)

    for train, test in tqdm(CV.split(X, y), total=CV.get_n_splits(X, y), desc='CV', leave=False):

        shap_model = pipe[2:].fit(X_conn[train], y[train])

        y_pred = shap_model.predict(X_conn[test])

        test_indices.extend(test)
        y_test_cv.append(y[test])
        y_pred_cv.append(y_pred)

        explainer = shap.Explainer(
            shap_model.predict, X_conn[train],
            feature_names=feature_names,
            # approximate=True,
            # model_output='raw',
            # feature_perturbation='interventional',
        )

        shap_values = explainer(X_conn[test], max_evals=2*len(feature_names) + 1)#, check_additivity=True)

        shap_values_cv.append(shap_values)

    # merge CV SHAPs

    # X = subjects.reshape(-1, 1)
    # X_test = pd.DataFrame(X[np.hstack(test_indices)], columns=['subject'])
    y_test = np.hstack(y_test_cv)
    y_pred = np.hstack(y_pred_cv)

    shap_values = shap.Explanation(
      values = np.vstack([sh.values for sh in shap_values_cv]),
      base_values = np.hstack([sh.base_values for sh in shap_values_cv]),
      data = np.vstack([sh.data for sh in shap_values_cv]),
      feature_names=feature_names,
      compute_time=np.sum([sh.compute_time for sh in shap_values_cv]),
      output_names=y_encoder.classes_,
      output_indexes=y_pred,
    )

    feature_dim_name = f'{"_".join(model_name.split("_")[0:2])}_feature'

    shap_ds = xr.Dataset({
      f'{model_name} shap': (('shap_dim', feature_dim_name), shap_values.values),
      f'{model_name} shap data': (('shap_dim', feature_dim_name), shap_values.data),
      f'{model_name} shap y_test': (('shap_dim'), y_encoder.inverse_transform(y_test)),
      f'{model_name} shap y_pred': (('shap_dim'), y_encoder.inverse_transform(y_pred)),
      },
      coords={feature_dim_name: feature_names}
    )

    shap_agg.append(shap_ds)

ds_shap = xr.merge(shap_agg)

In [ ]:
# LEARNING CURVE ANALYSIS (HOW DOES TRAIN/TRAIN SIZE IMPACT ACCURACY?)
# Note: this only analyze the best model

train_sizes, train_scores, test_scores = learning_curve(grid.best_estimator_, X, y,
                                                        cv=CV,
                                                        scoring='accuracy',
                                                        n_jobs=-1,
                                                        shuffle=True,
                                                        train_sizes=np.array([16, 18, 20, 22, 24]))


learning_curve_results = pd.DataFrame({
    'learning_curve_train_size': train_sizes,
    'learning_curve_mean_train_score': train_scores.mean(axis=1),
    'learning_curve_mean_test_score': test_scores.mean(axis=1)
})

learning_curve_results.index.name  = 'learning_curve_num'

ds_learning_curve = learning_curve_results.to_xarray()

In [ ]:
# STORE RESULTS

datasets = [
    {'X': xr.DataArray(X.flatten(), dims=['subject'])},
    {'y': xr.DataArray(y_encoder.inverse_transform(y), dims='subject')},
    {'y_classes': y_encoder.classes_},
    ds_grid,
    ds_learning_curve,
    ds_perm_test,
    ds_perm_importance]

if ENABLE_SHAP:
    datasets.append(ds_shap)

results = xr.merge(datasets)

with open(OUTPUT_PATH, 'wb') as f:
    results.to_netcdf(f, engine='h5netcdf')
    results.close()

# reload from disk
results = xr.open_dataset(OUTPUT_PATH, engine='scipy').load()

results